In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from django.contrib.gis.geos import Point, LineString as DjangoLineString
import pytz

In [2]:
from simple_history.utils import bulk_create_with_history
from helpers.histories import bulk_update_with_history
from openpyxl_image_loader import SheetImageLoader

In [3]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )
#     print(road_set)

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def get_connected_reference(company, resource, key, value):
    value=value.strip()
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [8]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    
    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [9]:
def get_sign(field, sign_field, occ, value):
    if sign_field == 'Cor':
        value=value.capitalize()
    elif sign_field == 'Tipo de Película':
        value=value.split(' ')[1]
    values = occ.form_fields['fields']
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [10]:
!cat .env

STAGE=PRODUCTION


In [11]:
filename = "Ocupacoes_SP 255 - rev 3"

In [12]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
# sheetname
ws = wb[sheetname]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [13]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
        
    values.append(obj)

In [14]:
company = Company.objects.get(name='Arteris Via Paulista - Faixa de Domínio')

occurrence_type=OccurrenceType.objects.get(uuid='762b144f-01f0-45c0-8b77-b4c831e04e42')
user=User.objects.get(username='rlcs')
utc = pytz.timezone('America/Sao_Paulo')
company,occurrence_type

(<Company: ec364a8e-b335-4729-8ac8-50ccabd73459: Arteris Via Paulista - Faixa de Domínio>,
 <OccurrenceType: Ocupações - ['Arteris Via Paulista - Faixa de Domínio']>)

In [15]:
for a in values:
    for c,v in a.items():
        if isinstance(v,str):
            a[c] = v.strip()
        if v == "Não possui" or v == "Não Possui":
            a[c]=''
        if c == 'Status do processos Artesp':
            a[c] = v.strip()
            if v == 'Autorizada' or v == 'Autorizada ' or v == 'Autorizada  ' or v == 'autorizada':
                a[c]='Autorizado'
            if v == 'Não autorizada' or v == 'Não Autorizada' or v == 'Não Autorizada ' or v == 'Não Autorizado':
                a[c]='Não autorizado'
            if v == 'Cancelada' or v == 'Cancelada ':
                a[c]='Cancelado'
            if v == 'Autorizado Anterior- NÃO IMPLANTADA':
                a[c]='Autorizado anterior'
        if c == 'Tipo da ocupação':
            if v == "Água e Esgoto":
                a[c] = 'ÁGUA/ESGOTO'
            if v == "Gasoduto":
                a[c] = 'Gasodutos'
        if c == 'Tipo Do Processo':
            if v == 'Não se Aplica' or v == 'Não se aplica':
                a[c] = ''
        if c == 'Lado Da Pista':
            if v == 'A identificar':
                a[c] = 'A Identificar'
        if c == 'Adimplente':
            if v == 'A identificar':
                a[c] = ''
        if c == 'Termo De Autorização':
            if v == 'A identificar':
                a[c] = ''
        if c == 'Validade Da Autorização':
            if v == 'A identificar':
                a[c] = ''
        if c == 'Lado Da Pista':
            if v == 'direito':
                a[c] = 'Direito'
        if c == 'Lado Da Pista':
            if v == 'Direito/Esquerdo':
                a[c] = 'Ambos'
        if c == 'Lado Da Pista':
            if v == 'esquerdo':
                a[c] = 'Esquerdo'
        if c == 'Tipo da ocupação':
            if v == 'Estação Radio Base':
                a[c] = 'Estação de Rádio Base'
        if c == 'Situação':
            if v == 'Não Autorizada':
                a[c] = 'Não autorizada'
            if v == 'Autorizado Anterior- NÃO IMPLANTADA':
                a[c]='Autorizado Anterior'
                
    a['Rodovia'] = a['Rodovia'].strip() if a['Rodovia'] is not None else None

In [16]:
for a in tqdm(values):
    a['Arr_anuidade'] = list(filter(None, [{'annuity_step': a['Número Anuidade_1'], 'annuity_begin_step': a['Início do período Anuidade_1'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Início do período Anuidade_1'] and a['Início do período Anuidade_1'] != ' ' else None, 'annuity_end_step': a['Fim do período Anuidade_1'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Fim do período Anuidade_1'] and a['Fim do período Anuidade_1'] != ' ' else None, 'annuity_value_step': a['Valor da anuidade [R$]_1'],'annuity_value_paid_step': a['Valor pago no ano [R$]_1']},
                                  {'annuity_step': a['Número Anuidade_2'], 'annuity_begin_step': a['Início do período Anuidade_2'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Início do período Anuidade_2'] and a['Início do período Anuidade_2'] != ' ' else None, 'annuity_end_step': a['Fim do período Anuidade_2'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Fim do período Anuidade_2'] and a['Fim do período Anuidade_2'] != ' ' else None, 'annuity_value_step': a['Valor da anuidade [R$]_2'],'annuity_value_paid_step': a['Valor pago no ano [R$]_2']},
                                  {'annuity_step': a['Número Anuidade_3'], 'annuity_begin_step': a['Início do período Anuidade_3'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Início do período Anuidade_3'] and a['Início do período Anuidade_3'] != ' ' else None, 'annuity_end_step': a['Fim do período Anuidade_3'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Fim do período Anuidade_3'] and a['Fim do período Anuidade_3'] != ' ' else None, 'annuity_value_step': a['Valor da anuidade [R$]_3'],'annuity_value_paid_step': a['Valor pago no ano [R$]_3']},
                                  {'annuity_step': a['Número Anuidade_4'], 'annuity_begin_step': a['Início do período Anuidade_4'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Início do período Anuidade_4'] and a['Início do período Anuidade_4'] != ' ' else None, 'annuity_end_step': a['Fim do período Anuidade_4'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Fim do período Anuidade_4'] and a['Fim do período Anuidade_4'] != ' ' else None, 'annuity_value_step': a['Valor da anuidade [R$]_4'],'annuity_value_paid_step': a['Valor pago no ano [R$]_4']},
                                  {'annuity_step': None, 'annuity_begin_step': None, 'annuity_end_step': None,'annuity_value_paid_step': a['Valor pago no ano [R$]_5']} if a['Valor pago no ano [R$]_5'] is not 0 else None ]))

  0%|          | 0/767 [00:00<?, ?it/s]

In [17]:
for a in tqdm(values):
    
    a['Notificacao'] = list(filter(None, [{'letter_number_notification_step': a['Número Carta Notificações_1'], 'data_notification_step': a['Data Notificações_1'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data Notificações_1'] != '' and a['Data Notificações_1'] != ' ' else None, 'ar_notification_step': a['Número Protocolo Notificações_1'], 'recebido_por_notification_step': a['Recebido Por Notificações_1']} if a['Data Notificações_1'] is not None else None,
                                  {'letter_number_notification_step': a['Número Carta Notificações_2'], 'data_notification_step': a['Data Notificações_2'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data Notificações_2'] != '' and a['Data Notificações_2'] != ' ' else None, 'ar_notification_step': a['Número Protocolo Notificações_2'], 'recebido_por_notification_step': a['Recebido Por Notificações_2']} if a['Data Notificações_2'] is not None else None]))

  0%|          | 0/767 [00:00<?, ?it/s]

In [18]:
values[4]

{'km': 83.52,
 'km final': 84.275,
 'km de Projeto': None,
 'km final de Projeto': None,
 'Status': None,
 'Equipe/Empreiteira': None,
 'Encontrado em': None,
 'Executado em': 'Não se Aplica',
 'Prazo': 'Não se Aplica',
 'Sentido': 'Não se Aplica',
 'Classe': 'Ocupações',
 'Faixa': None,
 'Pista': 'Tronco Simples',
 'km de referência': None,
 'Ramo': None,
 'Rodovia': 'SP 255',
 'Id': 7,
 'Pia': 'Não se aplica',
 'Pvpt': '2019-PVPT.015',
 'Lote': 29,
 'Especificado No Edital': 'Não',
 'Identificação Da Ocupação': 'Concessionária',
 'Data Da Autorização': '',
 'Interessado': 'Companhia Paulista de Força e Luz (CPFL)',
 'Número Protocolo': '',
 'Situação': 'A identificar',
 'Tipo da ocupação': 'Energia Elétrica',
 'Norma Der': 'DE 06/AFD-004',
 'Número Expediente': '',
 'Regional': 4,
 'Unidade': 'm',
 'Extensão': 755,
 'Tipo Do Processo': 'Regularização',
 'Latitude': 'A identificar',
 'Longitude': 'A identificar',
 'Contrato': '',
 'Lado Da Pista': 'Esquerdo',
 'Tipo de implantação': '

In [19]:
objects=[]
rep={}
problema=[]
for index, a in enumerate(tqdm(values)):

    point, road = get_road_coordinates(a['Rodovia'].replace(' ','-').strip(), a['km'], str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize() if a['Sentido'] is not None else 'Não se Aplica')) , company)
    if road == None:
        print('Linha:',index+2,a['Rodovia'].replace(' ','-'),'km:',a['km'])
        problema.append(['Linha:',index+2,a['Rodovia'].replace(' ','-'),a['km']])
        continue
#         road_manipulada = a['Código Do Acesso'].split(' - ')[0].replace(' ','-')
#         point, road = get_road_coordinates(road_manipulada, a['km'], str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize())) , company)
    try:
        rep={
                    "company":company,
                    "occurrence_type":occurrence_type,
                    "lane":get_connected_reference(company, 'reporting', 'lane', a['Faixa']) if a['Faixa'] is not None else get_connected_reference(company, 'reporting', 'lane', 'Não se aplica'),
                    "road":road,
                    "road_name":road.name,
                    "direction":get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize() if a['Sentido'] is not None else 'Não se Aplica'),
                    "created_by":user,
                    "found_at":datetime.datetime(2023, 5, 24, 0, 0).replace(tzinfo=utc),
                    "km":a['km'],
                    "end_km_manually_specified": True,
                    "end_km":a['km final'] if a['km final'] != 'A identificar' else None,
                    "point":point,
                    "track":'1',
                    "form_data":{
                        'id': a['Id'],
                        'pia': a['Pia'],
                        'pvpt': a['Pvpt'],
                        'lote': 29,
                        'especificado_no_edital': True if a['Especificado No Edital'].lower() == "sim" else False,
                        'identificacao_da_ocupacao': get_value('Identificação Da Ocupação',occurrence_type,a['Identificação Da Ocupação']) if 'Identificação Da Ocupação' in a else None,
                        'data_da_autorizacao': a['Data Da Autorização'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data Da Autorização'] is not None and a['Data Da Autorização'] != '' and a['Data Da Autorização'] != 'Não possui' else None,
                        'interessado': a['Interessado'],
                        'numero_protocolo': str(a['Número Protocolo']),
                        'situacao': get_value('Situação',occurrence_type,a['Situação'].capitalize().rstrip() if a['Situação'] != 'Autorizado Anterior' else 'Autorizado Anterior') if 'Situação' in a else None,
                        'modalidade': get_value('Tipo da ocupação',occurrence_type,a['Tipo da ocupação'].upper()) if 'Tipo da ocupação' in a else None,
                        'norma_der': get_value('Norma Der',occurrence_type,a['Norma Der']) if 'Norma Der' in a and a['Norma Der'] !='-' else None,
                        'numero_expediente': str(a['Número Expediente']) if a['Número Expediente'] else None,
                        'regional': [get_value('Regional',occurrence_type,str(float(a['Regional'])))] if str(a['Regional']) and a['Regional'] is not None else None,
                        'unidade': get_value('Unidade',occurrence_type,a['Unidade']) if a['Unidade'] else None,
                        'length': float(a['Extensão']),
                        'tipo_do_processo': get_value('Tipo Do Processo',occurrence_type,a['Tipo Do Processo']) if a['Tipo Do Processo'] != '' and a['Tipo Do Processo'] != None else None,
                        'latitude': a['Latitude'],
                        'longitude': a['Longitude'],
                        'contrato': a['Contrato'],
                        'lado_da_pista': get_value('Lado Da Pista',occurrence_type,a['Lado Da Pista'].strip()) if 'Lado Da Pista' in a and a['Lado Da Pista'] is not None else None,
                        'direcao': get_value('Tipo de implantação',occurrence_type,a['Tipo de implantação'].strip().upper()) if 'Tipo de implantação' in a and a['Tipo de implantação'] is not None else None,
                        'tipo': get_value('Tipo de passagem',occurrence_type,a['Tipo de passagem'].upper().strip()) if 'Tipo de passagem' in a and a['Tipo de passagem'] is not None else None,
                        'metodo': get_value('Método',occurrence_type,a['Método'].strip().upper()) if 'Método' in a and a['Método'] is not None else None,
                        'tep': None,
                        
                        'annuity_arr': a['Arr_anuidade'],
                        'annuity': a['Anuidade'],
                        'total_acumulado': a['Total Pago Acumulado'],
                        'adimplente': get_value('Adimplente',occurrence_type,a['Adimplente'].strip()) if a['Adimplente'] and a['Adimplente'] != 'A identificar' else None,
                        'termo_de_autorizacao': a['Termo De Autorização'],
                        'validade_da_autorizacao': a['Validade Da Autorização'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Validade Da Autorização'] is not None and a['Validade Da Autorização'] != '' and a['Validade Da Autorização'] != 'A Identificar' and a['Validade Da Autorização'] != 'Não possui' else None,
                        'termo_der': True if a['Cópia Do Termo Do Der'].lower() == "sim" else False,
                        'n_protocolo_artesp': a['Nº Protocolo Artesp'],
                        'n_processo_artesp': a['Nº Processo Artesp'],
                        'termo_autorizacao_artesp': a['Termo De Autorização Artesp'],
                        'status_artesp': get_value('Status do processos Artesp',occurrence_type,a['Status do processos Artesp'].strip()) if 'Status do processos Artesp' in a else None,
                        'data_status_artesp': a['Data Status'].replace(tzinfo=utc).strftime('%Y-%m-%dT03:00:00.000Z') if a['Data Status'] is not None and a['Data Status'] != '-' else None,
                        'notification': a['Notificacao'],
                        'interferencia': a['Interferência'],
                        'foi_remanejado': True if a['Foi Remanejado'].lower() == "sim" else False,
                        'data_remanejamento': a['Data Remanejamento'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data Remanejamento'] is not None and a['Data Remanejamento'] != '' and a['Data Remanejamento'] != 'Não Possui' else None,
                        'projeto_base': a['Projeto Base'] if a['Projeto Base'] != 'Não Possui' else None,
                        'rev_projeto': a['Rev. Projeto'] if a['Rev. Projeto'] != 'Não Possui' else None,
                        'notes': a['Observações'],
                        }
        }
#         if rep['form_data']['notification'] == []:
            
#             del rep['form_data']['notification']
            
#         if rep['form_data']['annuity_arr'] == []:
#             del rep['form_data']['annuity_arr']
        objects.append(Reporting(**rep))
    except Exception as e:
#         exception_type, exception_object, exception_traceback = sys.exc_info()
#         line_number = exception_traceback.tb_lineno
#         if a["Rodovia"] == 'SP-310':
#             print(f'AQUI: Km não existe no sistema: {a["km"]} | Rodovia não existe no sistema: {a["Rodovia"]} | Linha da Planilha: {index+2} | {road} | {road_manipulada}')
#             continue

        problema.append([index+2,a['Rodovia'].replace(' ','-'),a['km'],a['Sentido'],e])    
        print(f'AQUI: {e}| Linha da Planilha: {index+2}{objects.form_data}')
        
        continue

  0%|          | 0/767 [00:00<?, ?it/s]

In [20]:
problema

[]

In [21]:
objects[114].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f1492580110>,
 'uuid': UUID('8175cc38-dad3-4821-9139-2325b327d1e7'),
 'number': '',
 'company_id': UUID('ec364a8e-b335-4729-8ac8-50ccabd73459'),
 'road_name': 'SP-255',
 'road_id': 1929,
 'km': 156.01,
 'end_km': 156.01,
 'km_reference': None,
 'project_km': 0,
 'project_end_km': None,
 'point': <Point object at 0x7f14925541c0>,
 'direction': '5',
 'lane': '99',
 'track': '1',
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': None,
 'occurrence_type_id': UUID('762b144f-01f0-45c0-8b77-b4c831e04e42'),
 'form_data': {'id': 120,
  'pia': 'Não se aplica',
  'pvpt': '2019-PVPT.015',
  'lote': 29,
  'especificado_no_edital': False,
  'identificacao_da_ocupacao': '1',
  'data_da_autorizacao': None,
  'interessado': 'Companhia Paulista de Força e Luz (CPFL)',
  'numero_protocolo': '',
  'situacao': '6',
  'modalidade': '1',
  'norma_der': '1',
  'numero_expediente': None,
  'regional': 

In [22]:
for reporting in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/767 [00:00<?, ?it/s]

'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04197
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04198
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04199
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04200
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04201
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04202
'NoneType' object has no attribute 'order'
Searching KM on direction...
Sear

Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04250
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04251
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04252
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04253
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04254
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04255
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on R

'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04304
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04305
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04306
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04307
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04308
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04309
'NoneType' object has no attribute 'order'
Searching KM on direction...
Sear

Found KM on Road 1929: SP-255
VPT-IT-2023.04357
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1929: SP-255
VPT-IT-2023.04358
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1929: SP-255
VPT-IT-2023.04359
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1929: SP-255
VPT-IT-2023.04360
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1929: SP-255
VPT-IT-2023.04361
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1929: SP-255
VPT-IT-2023.04362
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1929: SP-255
VPT-IT-2023.04363
'NoneType' object has no att

...

VPT-IT-2023.04733
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04734
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04735
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04736
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04737
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04738
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04739
'NoneType' object has no attribute 'order'
Searching KM on

VPT-IT-2023.04787
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04788
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04789
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04790
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04791
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04792
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04793
'NoneType' object has no attribute 'order'
Searching KM on

Searching KM on direction...
Searching KM without direction...
Found KM on Road 1955: SP-249
VPT-IT-2023.04841
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04842
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04843
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04844
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04845
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04846
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on R

VPT-IT-2023.04894
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04895
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04896
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04897
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04898
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04899
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04900
'NoneType' object has no attribute 'order'
Searching KM on

VPT-IT-2023.04948
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04949
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04950
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04951
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04952
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04953
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.04954
'NoneType' object has no attribute 'order'
Searching KM on

In [47]:
# for r in Reporting.objects.filter(occurrence_type__occurrence_kind='2',created_at__gte='2023-5-3',company=company):
#     for a in values:
#         if r.form_data['id'] == a['Id']:
#             r.end_km_manually_specified = True
#             r.end_km = float(a['km final'])
#             r.save()
#             print(r.end_km, a['km final'],r.end_km_manually_specified)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.created_at received a naive datetime (2023-05-03 00:00:00) while time zone support is active.
  RuntimeWarning)


KeyError: 'id'

In [123]:
reporting.__dict__

{'_state': <django.db.models.base.ModelState at 0x7fe1fc45a810>,
 'uuid': UUID('62a575b5-e041-4437-8f13-47a64ea3cb77'),
 'number': 'VPT-IT-2023.00036',
 'company_id': UUID('ec364a8e-b335-4729-8ac8-50ccabd73459'),
 'road_name': 'SP-257',
 'road_id': 1920,
 'km': 15.0526,
 'end_km': 15.0526,
 'km_reference': None,
 'project_km': 0,
 'project_end_km': 0,
 'point': <Point object at 0x7fe1f96505b0>,
 'direction': '5',
 'lane': '99',
 'track': 1,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': None,
 'occurrence_type_id': UUID('762b144f-01f0-45c0-8b77-b4c831e04e42'),
 'form_data': {'id': 2605,
  'pia': 'Não se aplica',
  'pvpt': None,
  'especificado_no_edital': 'Não',
  'identificacao_da_ocupacao': '1',
  'data_da_autorizacao': None,
  'interessado': 'ViaPaulista S.A.',
  'numero_protocolo': 'None',
  'situacao': '6',
  'modalidade': '9',
  'norma_der': '4',
  'numero_expediente': None,
  'regional': '1',
  'unidade': '1',
  'leng

In [136]:
rep = Reporting.objects.get(number='VPT-IT-2023.00005')

In [139]:
rep.__dict__

{'_state': <django.db.models.base.ModelState at 0x7fe1f8fea650>,
 'uuid': UUID('940414fa-594a-436a-a0ff-96be7f4b3f28'),
 'number': 'VPT-IT-2023.00005',
 'company_id': UUID('ec364a8e-b335-4729-8ac8-50ccabd73459'),
 'road_name': 'SP-255',
 'road_id': 1919,
 'km': 48.0,
 'end_km': 48.0,
 'km_reference': None,
 'project_km': 0.0,
 'project_end_km': 0.0,
 'point': <Point object at 0x7fe1fa9ca880>,
 'direction': '0',
 'lane': '1',
 'track': '1',
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': None,
 'occurrence_type_id': UUID('762b144f-01f0-45c0-8b77-b4c831e04e42'),
 'form_data': {'lote': '29',
  'latitude': None,
  'longitude': None,
  'termo_der': 'Sim',
  'adimplente': 'Sim',
  'foi_remanejado': 'Sim',
  'total_acumulado': '{ "reduce": [{ "var": "formData.annuity" }, { "+": [{ "var": annuityValuePaidStep" }, { "var": "accumulator" }] }, 0 ] }',
  'especificado_no_edital': 'Sim'},
 'form_metadata': {'lote': {'manually_specified':